In [ ]:
import sqlite3
import time

# ----------------------------
# Connect to SQLite (in-memory DB for Colab)
# ----------------------------
conn = sqlite3.connect(":memory:")  # in-memory database
cursor = conn.cursor()
print("Connected to SQLite in-memory database successfully!\n")

# ----------------------------
# Create schema (similar to your MySQL DB)
# ----------------------------
cursor.executescript("""
CREATE TABLE Department (
    dept_id INTEGER PRIMARY KEY AUTOINCREMENT,
    dept_name TEXT UNIQUE NOT NULL
);

CREATE TABLE Member (
    member_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT UNIQUE,
    role TEXT CHECK(role IN ('Student','Faculty')) NOT NULL,
    dept_id INTEGER,
    FOREIGN KEY (dept_id) REFERENCES Department(dept_id)
);

CREATE TABLE Category (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT
);

CREATE TABLE Paper (
    paper_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    abstract TEXT,
    published_year INTEGER,
    dept_id INTEGER,
    category_id INTEGER,
    upload_date TEXT,
    pdf_link TEXT,
    FOREIGN KEY (dept_id) REFERENCES Department(dept_id),
    FOREIGN KEY (category_id) REFERENCES Category(category_id)
);

CREATE TABLE PaperAuthor (
    paper_id INTEGER,
    member_id INTEGER,
    PRIMARY KEY (paper_id, member_id),
    FOREIGN KEY (paper_id) REFERENCES Paper(paper_id),
    FOREIGN KEY (member_id) REFERENCES Member(member_id)
);

CREATE TABLE Keyword (
    keyword_id INTEGER PRIMARY KEY AUTOINCREMENT,
    keyword TEXT UNIQUE
);

CREATE TABLE PaperKeyword (
    paper_id INTEGER,
    keyword_id INTEGER,
    PRIMARY KEY (paper_id, keyword_id),
    FOREIGN KEY (paper_id) REFERENCES Paper(paper_id),
    FOREIGN KEY (keyword_id) REFERENCES Keyword(keyword_id)
);

CREATE TABLE Comment (
    comment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    paper_id INTEGER,
    member_id INTEGER,
    comment_text TEXT,
    timestamp TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (paper_id) REFERENCES Paper(paper_id),
    FOREIGN KEY (member_id) REFERENCES Member(member_id)
);

CREATE TABLE Review (
    review_id INTEGER PRIMARY KEY AUTOINCREMENT,
    paper_id INTEGER,
    faculty_id INTEGER,
    score INTEGER CHECK(score BETWEEN 1 AND 10),
    feedback TEXT,
    review_date TEXT,
    FOREIGN KEY (paper_id) REFERENCES Paper(paper_id),
    FOREIGN KEY (faculty_id) REFERENCES Member(member_id)
);
""")
print("Schema created successfully!\n")

# ----------------------------
# Insert initial sample data
# ----------------------------
cursor.execute("INSERT INTO Department (dept_name) VALUES ('Computer Science');")
cursor.execute("INSERT INTO Member (name, email, role, dept_id) VALUES ('Alice', 'alice@uni.edu', 'Student', 1);")
cursor.execute("INSERT INTO Member (name, email, role, dept_id) VALUES ('Bob', 'bob@uni.edu', 'Faculty', 1);")
cursor.execute("INSERT INTO Paper (title, abstract, published_year, dept_id, category_id) VALUES ('Sample Paper', 'Abstract here', 2025, 1, NULL);")
cursor.execute("INSERT INTO Review (paper_id, faculty_id, score, feedback, review_date) VALUES (1, 2, 7, 'Good work', '2025-12-09');")
conn.commit()

# ------------------------------------------------------
# 1. Atomicity Example
# ------------------------------------------------------
print("=== Atomicity Example ===")
cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
print("Before update:", cursor.fetchone()[0])

# Temporary update
conn.execute("BEGIN TRANSACTION;")
cursor.execute("UPDATE Paper SET title='Atomic Demo Temporary' WHERE paper_id=1;")
cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
print("After temporary update:", cursor.fetchone()[0])
conn.rollback()  # rollback
cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
print("After rollback:", cursor.fetchone()[0])

# Permanent update
conn.execute("BEGIN TRANSACTION;")
cursor.execute("UPDATE Paper SET title='Atomic Demo Committed' WHERE paper_id=1;")
conn.commit()
cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
print("After commit:", cursor.fetchone()[0], "\n")

# ------------------------------------------------------
# 2. Consistency Example
# ------------------------------------------------------
print("=== Consistency Example ===")
cursor.execute("SELECT COUNT(*) FROM Comment;")
before = cursor.fetchone()[0]
print("Before insert:", before)

conn.execute("BEGIN TRANSACTION;")
cursor.execute("""
INSERT INTO Comment (paper_id, member_id, comment_text)
SELECT 1, 1, 'Consistency Temp'
WHERE NOT EXISTS (SELECT 1 FROM Comment WHERE comment_text='Consistency Temp');
""")
cursor.execute("SELECT COUNT(*) FROM Comment;")
print("After temporary insert:", cursor.fetchone()[0])
conn.rollback()
cursor.execute("SELECT COUNT(*) FROM Comment;")
print("After rollback:", cursor.fetchone()[0])

# Permanent insert
conn.execute("BEGIN TRANSACTION;")
cursor.execute("""
INSERT INTO Comment (paper_id, member_id, comment_text)
SELECT 1, 1, 'Consistency Committed'
WHERE NOT EXISTS (SELECT 1 FROM Comment WHERE comment_text='Consistency Committed');
""")
conn.commit()
cursor.execute("SELECT COUNT(*) FROM Comment;")
print("After commit:", cursor.fetchone()[0], "\n")

# ------------------------------------------------------
# 3. Isolation Example (SQLite is serialized by default)
# ------------------------------------------------------
print("=== Isolation Example ===")
cursor.execute("SELECT score FROM Review WHERE review_id=1;")
original_score = cursor.fetchone()[0]
print("Before temporary update:", original_score)

conn.execute("BEGIN TRANSACTION;")
cursor.execute("UPDATE Review SET score=? WHERE review_id=1;", (original_score + 2,))
cursor.execute("SELECT score FROM Review WHERE review_id=1;")
print("After temporary update:", cursor.fetchone()[0])
conn.rollback()
cursor.execute("SELECT score FROM Review WHERE review_id=1;")
print("After rollback:", cursor.fetchone()[0])

# Permanent update
conn.execute("BEGIN TRANSACTION;")
cursor.execute("UPDATE Review SET score=? WHERE review_id=1;", (original_score + 3,))
conn.commit()
cursor.execute("SELECT score FROM Review WHERE review_id=1;")
print("After commit:", cursor.fetchone()[0], "\n")

# ------------------------------------------------------
# 4. Durability Example
# ------------------------------------------------------
print("=== Durability Example ===")
cursor.execute("SELECT COUNT(*) FROM Member;")
original_count = cursor.fetchone()[0]
print("Original Member count:", original_count)

conn.execute("BEGIN TRANSACTION;")
cursor.execute("INSERT INTO Member (name, email, role, dept_id) VALUES ('Durable Temp', 'temp@uni.edu', 'Student', 1);")
cursor.execute("SELECT COUNT(*) FROM Member;")
print("After temporary insert:", cursor.fetchone()[0])
conn.rollback()
cursor.execute("SELECT COUNT(*) FROM Member;")
print("After rollback:", cursor.fetchone()[0])

# Permanent insert
unique_email = f"commit_{int(time.time())}@uni.edu"
conn.execute("BEGIN TRANSACTION;")
cursor.execute("INSERT INTO Member (name, email, role, dept_id) VALUES (?, ?, 'Student', 1);", ('Durable Commit', unique_email))
conn.commit()
cursor.execute("SELECT COUNT(*) FROM Member;")
print("After commit:", cursor.fetchone()[0], "\n")

# ------------------------------------------------------
# 5. Indexing Demonstration
# ------------------------------------------------------
print("=== Indexing Demonstration ===")
cursor.execute("""
CREATE TABLE IF NOT EXISTS LargeTable (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    score INTEGER,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
);
""")
conn.commit()

# Insert 5000 rows if table is empty
cursor.execute("SELECT COUNT(*) FROM LargeTable;")
if cursor.fetchone()[0] < 5000:
    print("Inserting 5000 rows...")
    for i in range(5000):
        cursor.execute("INSERT INTO LargeTable (name, score) VALUES (?, ?);", (f"Name{i}", i % 100))
    conn.commit()
    print("Data inserted.\n")

# Query without index
start = time.time()
cursor.execute("SELECT * FROM LargeTable WHERE score = 50;")
rows = cursor.fetchall()
print(f"Query without index returned {len(rows)} rows in {time.time() - start:.5f} seconds")

# Create index (ignore if already exists)
try:
    cursor.execute("CREATE INDEX idx_score ON LargeTable(score);")
    conn.commit()
    print("Index created successfully!")
except sqlite3.OperationalError as err:
    if "already exists" in str(err):
        print("Index already exists — skipping.")
    else:
        raise

# Query with index
start = time.time()
cursor.execute("SELECT * FROM LargeTable WHERE score = 50;")
rows = cursor.fetchall()
print(f"Query with index returned {len(rows)} rows in {time.time() - start:.5f} seconds")

# ----------------------------
# Close connection
# ----------------------------
cursor.close()
conn.close()
print("SQLite connection closed.")


Connected to SQLite in-memory database successfully!

Schema created successfully!

=== Atomicity Example ===
Before update: Sample Paper
After temporary update: Atomic Demo Temporary
After rollback: Sample Paper
After commit: Atomic Demo Committed 

=== Consistency Example ===
Before insert: 0
After temporary insert: 1
After rollback: 0
After commit: 1 

=== Isolation Example ===
Before temporary update: 7
After temporary update: 9
After rollback: 7
After commit: 10 

=== Durability Example ===
Original Member count: 2
After temporary insert: 3
After rollback: 2
After commit: 3 

=== Indexing Demonstration ===
Inserting 5000 rows...
Data inserted.

Query without index returned 50 rows in 0.00061 seconds
Index created successfully!
Query with index returned 50 rows in 0.00038 seconds
SQLite connection closed.
